In [1]:
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from scipy import interpolate
from matplotlib.pyplot import imshow
import math
import sys
%matplotlib inline
np.set_printoptions(threshold=sys.maxsize)
from collections import Counter
import scipy
import pandas as pd

import pydicom
import png

import shutil
import xlrd as x
import png
import pydicom

In [2]:
! pip install dicom

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
! pip install pypng

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
class GrayscaleImage(object):
    
    def __init__(self, image, width, height):
        self.image = image
        self.width = width
        self.height = height
    
    def __str__(self):
          return '[%dx%d]' % (self.width, self.height)


def shade_at_percentile(hist, percentile):
    """ Assumes the argument percentile,
        to be less 1.
    """
    n = np.sum(hist)
    cumulative_sum = np.cumsum(hist)
    
    return np.argmax(cumulative_sum/n >= percentile)

# from contrast.py
import numpy as np

def shade_at_percentile(hist, percentile):
    """ Assumes the argument percentile,
        to be less 1.
    """
    n = np.sum(hist)
    cumulative_sum = np.cumsum(hist)
    
    return np.argmax(cumulative_sum/n >= percentile)

def auto_contrast(image):
    """ Apply auto contrast to an image using
        https://stackoverflow.com/questions/9744255/instagram-lux-effect/9761841#9761841
    """
    hist, _ = np.histogram(image.image.ravel(), bins=np.arange(0, 256))
    p01 = shade_at_percentile(hist, .01)
    p99 = shade_at_percentile(hist, .99)
    a = 255.0/(p99 + p01)
    b = -1.0 * a * p01

    result = (image.image.astype(float) * a) + b
    result = result.clip(0, 255.0)
    
    return GrayscaleImage(np.uint8(result), image.width, image.height)


# from init.py
import os
#import png
import pydicom
import numpy as np


def mri_to_png(mri_file, png_file, do_auto_contrast=False):
    """ Function to convert from a DICOM image to png
        @param mri_file: An opened file like object to read te dicom data
        @param png_file: An opened file like object to write the png data
    """

    image_2d = extract_grayscale_image(mri_file)

    if do_auto_contrast:
        image_2d = auto_contrast(image_2d)

    # Writing the PNG file
    w = png.Writer(image_2d.width, image_2d.height, greyscale=True)
    w.write(png_file, image_2d.image)

def extract_grayscale_image(mri_file):
    # Extracting data from the mri file
    plan = pydicom.read_file(mri_file)
    shape = plan.pixel_array.shape

    #Convert to float to avoid overflow or underflow losses.
    image_2d = plan.pixel_array.astype(float)

    # Rescaling grey scale between 0-255
    image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0
    
    #Convert to uint
    image_2d_scaled = np.uint8(image_2d_scaled)

    return GrayscaleImage(image_2d_scaled, shape[1], shape[0])


def convert_file(mri_file_path, png_file_path, auto_contrast=False):
    """ Function to convert an MRI binary file to a
        PNG image file.
        @param mri_file_path: Full path to the mri file
        @param png_file_path: Fill path to the png file
    """

    # Making sure that the mri file exists
    if not os.path.exists(mri_file_path):
        raise Exception('Source file "%s" does not exists' % mri_file_path)

    # Making sure the png file does not exist
    if os.path.exists(png_file_path):
        print('Removing existing output file %s' % png_file_path)
        os.remove(png_file_path)

    mri_file = open(mri_file_path, 'rb')
    png_file = open(png_file_path, 'wb')

    mri_to_png(mri_file, png_file, auto_contrast)

    png_file.close()




In [11]:
folder_dcm = "/media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST/"
folder_png = "/media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/"


In [12]:
convert_file(folder_dcm+'1.dcm',folder_dcm+'1.png')

In [19]:
for file in os.listdir(folder_dcm):
    #print file.split('.')[0]+'.png'
    if file.endswith('.dcm'):
        print file
        convert_file(folder_dcm+file,folder_png+file.split('.')[0]+'.png')

135.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/135.png
479.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/479.png
0.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/0.png
1.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1.png
10.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/10.png
100.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/100.png
1000.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1000.png
1001.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1001.png
1002.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1002.png
1003.dcm
Removing exist

1164.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1164.png
1165.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1165.png
1166.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1166.png
1167.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1167.png
1168.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1168.png
1169.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1169.png
1015.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1015.png
1033.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1033.png
1051.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1051.png
1

185.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/185.png
186.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/186.png
187.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/187.png
188.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/188.png
189.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/189.png
19.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/19.png
190.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/190.png
191.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/191.png
193.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/193.png
194.dcm
Removing exis

257.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/257.png
258.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/258.png
259.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/259.png
26.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/26.png
260.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/260.png
261.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/261.png
262.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/262.png
263.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/263.png
264.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/264.png
265.dcm
Removing exis

349.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/349.png
35.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/35.png
350.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/350.png
351.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/351.png
352.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/352.png
353.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/353.png
354.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/354.png
355.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/355.png
356.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/356.png
357.dcm
Removing exis

...

819.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/819.png
838.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/838.png
857.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/857.png
876.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/876.png
895.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/895.png
913.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/913.png
932.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/932.png
951.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/951.png
970.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/970.png
877.dcm
Removing ex

940.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/940.png
941.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/941.png
942.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/942.png
943.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/943.png
944.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/944.png
945.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/945.png
946.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/946.png
947.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/947.png
948.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/948.png
949.dcm
Removing ex

1028.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1028.png
1029.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1029.png
103.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/103.png
1030.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1030.png
1031.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1031.png
1032.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1032.png
1034.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1034.png
1035.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1035.png
1036.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1036.png
103

1099.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1099.png
11.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/11.png
110.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/110.png
1100.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1100.png
1101.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1101.png
1102.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1102.png
1103.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1103.png
1104.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1104.png
1106.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1106.png
1107.dc

1188.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1188.png
1189.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1189.png
119.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/119.png
1190.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1190.png
1191.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1191.png
1192.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1192.png
1193.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1193.png
1194.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1194.png
1195.dcm
Removing existing output file /media/integral-m/TOSHIBA EXT/dataset_fluro_onko/ONKO_TEST_PNG/1195.png
119